In [1]:
import sys

sys.path.append(r"D:\VENV_PYTHON\py3_8\Scripts")
from my_packages import *

# set the maximum number of rows and columns to display
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 300)

In [2]:
def export_parameter_record(savefolder, params_dict):

    import os

    for i in range(1, 1000):
        savepath = os.path.join(savefolder, "Param_Record_{}.txt".format(str(i).zfill(3)))
        if os.path.exists(savepath):
            continue
        else:
            break

    with open(savepath, "w+") as file:
        for key, value in params_dict.items():
            file.write("{} : {}".format(key, value))
            file.write("\n\n")

    pop_up_string = "Saved at {}".format(savefolder)
    print(pop_up_string)


def open_geodata(filepath):
    import sys

    extension = filepath.split(".")[-1]

    if extension == "pkl":
        geodata = pd.read_pickle(filepath, compression="zip")
    elif extension == "shp":
        geodata = gpd.read_file(filepath)
    else:
        sys.exit(
            "The datatype of PSC_filepath is invalid.\
        \nExpect {} or {}, not {}".format(
                type("1"), geodata_type, type(PSC_filepath)
            )
        )
    return geodata

#### Data Input

In [3]:
# profile_segment_psc_folder = r"E:\012_HOPE_TO_PUBLISH_PAPER\PROCESS_023\15_CALIBRATION_v003\CALIBRATED_Displacement_Vertical_mm\v004\HSR_PROFILE\AlongTrackProfile\PSC_GeoData"
profile_segment_psc_folder = r"D:\SENTINEL_1A_SAR_IMAGES\030_REPROCESS_CHOUSHUI_P2\CALIBRATION_PROCESS\Calibrated_Files\LOS_mm_ALL_POINTS_v010\VERT_mm\HSR_PROFILE\AlongTrackProfile"

file2process = glob(os.path.join(profile_segment_psc_folder, "Output_*"))

# savefolder = r"E:\012_HOPE_TO_PUBLISH_PAPER\PROCESS_023\15_CALIBRATION_v003\CALIBRATED_Displacement_Vertical_mm\v004\HSR_PROFILE\AlongTrackProfile\PSC_CSV_Export"

savefolder = r"D:\SENTINEL_1A_SAR_IMAGES\030_REPROCESS_CHOUSHUI_P2\CALIBRATION_PROCESS\Calibrated_Files\LOS_mm_ALL_POINTS_v010\VERT_mm\HSR_PROFILE\AlongTrackProfile"

if not os.path.exists(savefolder):
    os.makedirs(savefolder)

params_dict = {
    "profile_segment_psc_folder": profile_segment_psc_folder,
    "file2process": file2process,
    "savefolder": savefolder,
}

#### Function Processing

In [4]:
for file in file2process:

    data = open_geodata(file)

    data = data.iloc[:, 3:-1]

    unique_segment = np.unique(data["Segment"].values)

    amount_list = []

    empty_mean = pd.DataFrame(data=None)
    empty_stdev = pd.DataFrame(data=None)

    for segment_name in unique_segment:

        data_by_segment = data[data["Segment"] == segment_name]
        average_by_segment = data_by_segment.groupby("Segment").mean()
        stdev_by_segment = data_by_segment.groupby("Segment").std()

        amount_list.append(len(data_by_segment))

        empty_mean = pd.concat([empty_mean, average_by_segment])
        empty_stdev = pd.concat([empty_stdev, stdev_by_segment])

    empty_mean["Pts_Num"] = amount_list
    empty_stdev["Pts_Num"] = amount_list

    savename_mean = "Mean_" + os.path.basename(file)[:-4] + ".xlsx"
    savename_stdev = "Stdev_" + os.path.basename(file)[:-4] + ".xlsx"

    empty_mean.to_excel(os.path.join(savefolder, savename_mean))
    empty_stdev.to_excel(os.path.join(savefolder, savename_stdev))

export_parameter_record(savefolder=savefolder, params_dict=params_dict)

Saved at D:\SENTINEL_1A_SAR_IMAGES\030_REPROCESS_CHOUSHUI_P2\CALIBRATION_PROCESS\Calibrated_Files\LOS_mm_ALL_POINTS_v010\VERT_mm\HSR_PROFILE\AlongTrackProfile
